# **Amazon Lookout for Equipment** - Time series annotation with LabelStudio

*Part 1 - Initialization*

## Initialization
---
### Imports

In [1]:
import boto3

from binascii import hexlify
from json import load
from os import urandom
from uuid import uuid4

### Helper functions

In [2]:
def get_notebook_name():
    log_path = '/opt/ml/metadata/resource-metadata.json'
    with open(log_path, 'r') as logs:
        _logs = load(logs)
    return _logs['ResourceName']

### Parameters
Run the following cell to collect the parameters needed to run the LabelStudio container:

In [4]:
notebook_name  = get_notebook_name()
session        = boto3.session.Session()
current_region = session.region_name

### Setup authentication
You will need to authenticate yourself against LabelStudio. Update the following cell before running it to setup your `user`, `password` and `token`:

In [3]:
# Must follow a valid email address format:
username = 'username@domain.com'

# Fill in a password for your LabelStudio user:
password = 'ChooseAPassword'

# You can uncomment the following line to generate a unique token:
# token = hexlify(urandom(20)).decode()
token = 'GenerateAnAPIToken'
print(token)

GenerateAnAPIToken


Let's store these variables in the Jupyter environment so that we can use them in the next notebook:

In [5]:
%store token
%store notebook_name
%store current_region

Stored 'token' (str)
Stored 'notebook_name' (str)
Stored 'current_region' (str)


## Run LabelStudio
---
Run the following cell to create a shell script which will download a Docker image for LabelStudio and run it:

In [7]:
with open('label-studio.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    f.write(f"""docker run \
    -it \
    -p 8080:8080 \
    -v /home/ec2-user/SageMaker/label-studio-data:/label-studio/data \
    heartexlabs/label-studio:latest label-studio start --username {username} --password {password} --user-token {token} \
    --host https://{notebook_name}.notebook.{current_region}.sagemaker.aws/proxy/8080""")

The following cell will run the script you just created and launch LabelStudio. Once you see the following message, you can cruise over the next notebook:

```
Django version 3.1.14, using settings 'core.settings.label_studio'
Starting development server at http://0.0.0.0:8080/
Quit the server with CONTROL-C.
```

The following cell will stay in a running state (note the `[*]` next to it) until you forcefully stop it (note that this will also stop LabelStudio):

In [8]:
!source ./label-studio.sh

./deploy/docker-entrypoint.sh: Looking for init scripts in /label-studio/deploy/docker-entrypoint.d/
./deploy/docker-entrypoint.sh: Launching /label-studio/deploy/docker-entrypoint.d/10-copy-static-data.sh
=> Skipping copy data.
./deploy/docker-entrypoint.sh: Launching /label-studio/deploy/docker-entrypoint.d/20-wait-for-db.sh
=> Skipping wait for database.
./deploy/docker-entrypoint.sh: Launching /label-studio/deploy/docker-entrypoint.d/30-run-db-migrations.sh
=> Skipping run db migrations.
./deploy/docker-entrypoint.sh: Configuration complete; ready for start up
=> Hostname correctly is set to: https://lookout-equipment.notebook.eu-west-1.sagemaker.aws/proxy/8080
=> Django URL prefix is set to: /proxy/8080
=> Database and media directory: /label-studio/data
=> Static URL is set to: /static/
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /pypi/label-studio/json HTTP/1.1" 200 46133
Initializing database..
Performing system checks...

System check identified n

## Cleanup
---
If you want to stop LabelStudio, just click on the `Interrupt the kernel` button in this notebooks toolbar.

If you don't want to keep your labeling projects with your ongoing label work, you can safely delete the `/home/ec2-user/SageMaker/label-studio-data` folder where all the label data is stored.

**Do not delete** this folder if you want to continue your labeling work later, or isn't done processing the labeling job outputs.